## 准备数据

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [2]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [3]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        ####################
        self.hidden_layer_size = 256
        self.input_size = 28 * 28
        self.output_size = 10
        self.W1 = tf.Variable(tf.random.uniform(shape=[self.input_size, self.hidden_layer_size], minval=-0.1, maxval=0.1))
        self.b1 = tf.Variable(tf.zeros(self.hidden_layer_size))
        self.W2 = tf.Variable(tf.random.uniform(shape=[self.hidden_layer_size, self.output_size], minval=-0.1, maxval=0.1))
        self.b2 = tf.Variable(tf.zeros(self.output_size))
    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        ####################

        x_flattened = tf.reshape(x, [-1, self.input_size])
        hidden_layer = tf.nn.relu(tf.matmul(x_flattened, self.W1) + self.b1)
        logits = tf.matmul(hidden_layer, self.W2) + self.b2
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [4]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        print(x.shape)
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [7]:
train_data, test_data = mnist_dataset()
for epoch in range(100):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 1.489975 ; accuracy 0.7148
epoch 1 : loss 1.48537 ; accuracy 0.71611667
epoch 2 : loss 1.4807819 ; accuracy 0.71735
epoch 3 : loss 1.4762108 ; accuracy 0.71875
epoch 4 : loss 1.4716566 ; accuracy 0.7198833
epoch 5 : loss 1.4671187 ; accuracy 0.72095
epoch 6 : loss 1.462598 ; accuracy 0.72215
epoch 7 : loss 1.458094 ; accuracy 0.7234
epoch 8 : loss 1.4536073 ; accuracy 0.72461665
epoch 9 : loss 1.4491378 ; accuracy 0.7255833
epoch 10 : loss 1.4446853 ; accuracy 0.7265667
epoch 11 : loss 1.44025 ; accuracy 0.72805
epoch 12 : loss 1.435832 ; accuracy 0.72898334
epoch 13 : loss 1.4314312 ; accuracy 0.73011667
epoch 14 : loss 1.4270476 ; accuracy 0.73075
epoch 15 : loss 1.4226812 ; accuracy 0.7317
epoch 16 : loss 1.4183323 ; accuracy 0.7327167
epoch 17 : loss 1.4140003 ; accuracy 0.7337833
epoch 18 : loss 1.4096862 ; accuracy 0.73478335
epoch 19 : loss 1.4053893 ; accuracy 0.73573333
epoch 20 : loss 1.4011102 ; accuracy 0.7367333
epoch 21 : loss 1.3968484 ; accuracy 0.7374167